In [3]:
# Choose variant and machine type
VARIANT = '7b-it-quant' #@param ['2b', '2b-it', '9b', '9b-it', '27b', '27b-it']
MACHINE_TYPE = 'cuda' #@param ['cuda', 'cpu']

CONFIG = VARIANT
if CONFIG == '7b-it-quant':
    CONFIG = '7b'

In [4]:
import torch
torch.cuda.empty_cache()

In [5]:
weights_dir = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

In [6]:
import os
# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-7b-it-quant.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

In [7]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 239 (delta 86), reused 58 (delta 55), pack-reused 116 (from 1)
Receiving objects: 100% (239/239), 2.18 MiB | 18.42 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [8]:
import sys

sys.path.append('gemma_pytorch')

In [9]:
from gemma.config import GemmaConfig, get_model_config
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch

In [10]:
# Set up model config.
model_config = get_model_config(CONFIG)
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT
torch.cuda.empty_cache()

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

In [13]:
import time

# Đo thời gian cho việc sinh mẫu
start_time = time.time()

sentence = """
    Trận Điện Biên Phủ, còn gọi là Chiến dịch Điện Biên Phủ là trận đánh lớn nhất trong Chiến tranh Đông Dương lần thứ nhất diễn ra tại lòng chảo Mường Thanh, châu Điện Biên, tỉnh Lai Châu (nay thuộc thành phố Điện Biên Phủ, tỉnh Điện Biên) giữa Quân đội Nhân dân Việt Nam và Quân đội Liên hiệp Pháp (gồm Lực lượng Viễn chinh Pháp, Binh đoàn Lê dương Pháp, quân phụ lực bản xứ và Quân đội Quốc gia Việt Nam). Đây là chiến thắng quân sự lớn nhất trong cuộc Chiến tranh Đông Dương (1945 – 1954) của Việt Nam. Với thắng lợi quyết định này, lực lượng Quân đội Nhân dân Việt Nam do Đại tướng Võ Nguyên Giáp chỉ huy đã buộc quân Pháp và quân Quốc Gia Việt Nam tại Điện Biên Phủ phải đầu hàng vào ngày 7 tháng 5 năm 1954, sau suốt 2 tháng cầm cự. Giữa trận này, quân Pháp đã gia tăng lên đến 16.200 người nhưng vẫn không thể cầm cự được trước các đợt tấn công như vũ bão của Quân đội Nhân dân Việt Nam. Thực dân Pháp đã không thể bình định Việt Nam bất chấp nhiều năm chiến đấu và với sự hỗ trợ và can dự ngày càng sâu của Hoa Kỳ. Pháp đã không còn bất kì khả năng nào để tiếp tục chiến đấu tại Việt Nam sau thảm bại này. Trên phương diện quốc tế, chiến dịch này có một ý nghĩa rất lớn và đã đi vào lịch sử nhân loại, bởi đây là lần đầu tiên quân đội của một nước từng là thuộc địa ở châu Á đánh bại đội quân hiện đại và tối tân của một cường quốc châu Âu, được hỗ trợ bởi đồng minh là Hoa Kỳ trong một chiến dịch quân sự lớn. Được xem là một thảm họa bất ngờ đối với thực dân Pháp và cũng là một đòn giáng mạnh với thế giới phương Tây, đã đánh bại ý chí duy trì thuộc địa Đông Dương của Pháp và buộc nước này phải hòa đàm và rút quân ra khỏi Đông Dương. Với niềm tin được cổ vũ mạnh mẽ bởi chiến thắng này, các thuộc địa của Pháp ở châu Phi cũng đồng loạt nổi dậy đòi độc lập. Chỉ riêng trong năm 1960, 17 nước châu Phi đã giành được độc lập và đến năm 1967, Pháp đã buộc phải trao trả độc lập cho hầu hết các nước từng là thuộc địa của họ.
"""
word_count = len(sentence.split(' '))
limit = 200

low_range = int(word_count * 0.5)
limit = low_range if low_range > limit else limit

prompt = f"""<start_of_turn> 
            Hãy tóm tắt đoạn văn sau
              {sentence} 
            Lưu ý:
                - Văn bản đã tóm tắt phải là đơn văn bản.
                - Văn bản đã tóm tắt phải được viết bằng ngôn ngữ tiếng Việt.
                - Văn bản đã tóm tắt phải được tổ chức theo định dạng Markdown, trong đó phải có
                  cụm '## Tóm tắt: ' ở đầu văn bản, văn bản đã tóm tắt phải được đặt sau cụm này.
                - Văn bản đã tóm tắt phải được chuẩn hoá, loại bỏ các khoảng trống dư thừa.
                - Chỉ tóm tắt ý, không thực hiện bất kỳ tác vụ nào khác.
                - Độ dài tối đa {limit} từ.
            <end_of_turn>
            """

# Generate sample
results = model.generate(
    prompt,
    device=device,
    output_len=limit,
)
print(results)
generation_time = time.time() - start_time
print(generation_time)

## Tóm tắt:

Trận Điện Biên Phủ là chiến thắng quân sự lớn nhất trong cuộc chiến tranh Đông Dương của Việt Nam. Với thắng lợi quyết định này, lực lượng Quân đội nhân dân Việt Nam đã buộc quân Pháp và quân Quốc Gia Việt Nam tại Điện Biên Phủ phải đầu hàng vào ngày 7 tháng 5 năm 1954. chiến dịch này có một ý nghĩa rất lớn và đã đi vào lịch sử nhân loại bởi đây là lần đầu tiên quân đội của một nước từng là thuộc địa ở châu Á đánh bại đội quân hiện đại và tối tân của một cường quốc châu Âu, được hỗ trợ bởi đồng minh là Hoa Kỳ trong một chiến dịch quân sự lớn
91.68055653572083


In [14]:
# Generate with one request in chat mode

# Chat templates
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn><eos>\n"
MODEL_CHAT_TEMPLATE = "<start_of_turn>model\n{prompt}<end_of_turn><eos>\n"

# Sample formatted prompt
prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt='What is a good place for travel in the US?'
    )
    + MODEL_CHAT_TEMPLATE.format(prompt='California.')
    + USER_CHAT_TEMPLATE.format(prompt='What can I do in California?')
    + '<start_of_turn>model\n'
)
print('Chat prompt:\n', prompt)

results = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=128,
)
print(results)

Chat prompt:
 <start_of_turn>user
What is a good place for travel in the US?<end_of_turn><eos>
<start_of_turn>model
California.<end_of_turn><eos>
<start_of_turn>user
What can I do in California?<end_of_turn><eos>
<start_of_turn>model

There's a lot to see and do in California, depending on your interests and preferences:

**Classic destinations:**
- **San Francisco:** Golden Gate Bridge, Alcatraz Island, Chinatown, Golden Gate Park
- **Los Angeles:** Hollywood Walk of Fame, Griffith Observatory, Santa Monica Pier, Rodeo Drive
- **San Diego:** Balboa Park, La Jolla Cove, Altoshahar National Monument
- **Lake Tahoe:** Beautiful mountain scenery, hiking trails, boating and water sports

**Other popular destinations:**
- Monterey and Carmel-by-the-Sea: Charming coastal towns
- Yosemite National Park: Majestic granite
